# CMSE 381 Final Project Template

**INSTRUCTIONS**: This is a template to help organize your project.  All projects should include the 5 major sections below (you do not need to use this template file).  If you use this file, complete your work below and remove content in parentheses. Also, remove this current cell.  

**Sarah***: Work on filling out project template as much as possible, Confusion Matrix per question 3 
**Thuy***: Split the files into X and Y, work on the SVM model per question 4
SELF DUE DATE: APRIL 7TH, meet in person, in wells hall 

Both: Work on K-Fold and subset selection per question 5 

#### CMSE 381 Final Project
### &#9989; Group members: Sarah Horan, Thuy Nguyen
### &#9989; Section_001
#### &#9989; 3/31/25

# ___PREDICTING FACE ORIETATION___

## Background and Motivation

How does the brain recognize not just who we’re looking at, but how they’re positioned in space? This question lies at the intersection of neuroscience and data science. In this project, we investigate how neurons in the anterior medial (AM) face patch of a macaque monkey respond when the animal views faces from different orientations. This brain region is known to be selective for facial features, and here we test whether its activity can reveal not just the presence of a face, but the **direction it's facing**.

The dataset we’re using comes from a study where a monkey passively viewed face images from 25 individuals, each shown in 8 different head orientations. For each image, researchers recorded binary neural spike data across a short time window (0–400 ms) following the image onset.

Our main goal is to explore whether we can **predict the orientation of a viewed face**—such as frontal, profile, or turned head—based on this neural activity alone. This is a classic example of a **neural decoding** task, where machine learning models are trained to extract meaning from complex biological signals.

Specifically, we aim to answer:
- **Can we classify face orientation based on patterns in neural spike data?**

By addressing these questions, we hope to gain insight into how the brain encodes spatial information about faces—and demonstrate how data science tools can uncover patterns in neural data that may not be obvious at first glance.


## Methodology
_(How did you go about answering your question(s)? You should wrote some code here to demonstrate what the data is like and how in principle your method works. You can leave the variations of the related to specific results to the results section.)_

In [ ]:
# you may want to import some modules here

### Data
_(Describe the data you are using. What variables are you using? What they mean? Why did you choose them?)_

In [ ]:
# you may need some code here to adjust the format or organization of your data so that they can be used for the model

In [1]:
# you may want to write some code to see what the data looks like

### Models for classification _(if applicable)_
_(What models will you be using for classification? Why did you choose to use them? What questions would you answer with them? How would you evaluate if each model? What cross-validation method did you use?)_

In [ ]:
# you may add some code here to show how the model works in principle

### Models for regression _(if applicable)_
_(What models will you be using for regression? Why did you choose to use them? What questions would you answer with them? How would you evaluate if each model? What cross-validation method did you use?)_

In [ ]:
# you may add some code here to show how the model works in principle

### Other methods used _(if applicable)_

_(If this is a preprocessing step to prepare your data for regression or classification models, you should put this subsection before your explanation for the regression or classification models.)_

_(What method did you use otherwise? Why did you choose to use them? What questions would you answer with them? How would you evaluate the results? What cross-validation method did you use when applicable?)_

In [ ]:
# you may add some code here to show how the method works in principle

# you may add some code here to show how the model works in principle

## Results

_(What did you find when you carried out your methods? Some of your code related to
presenting results/figures/data may be replicated from the methods section or may only be present in
this section. All of the plots that you plan on using for your presentation should be present in this
section)_

### classification results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

### regression results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

### other results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

## Discussion and Conclusion

_(What did you learn from your results? What obstacles did you run into? What would you do differently next time? Clearly provide quantitative answers to your question(s)?  At least one of your questions should be answered with numbers.  That is, it is not sufficient to answer "yes" or "no", but rather to say something quantitative such as variable 1 increased roughly 10% for every 1 year increase in variable 2.)_

### discussion on the classification results

### discussion on the regression results

### discussion on the other results

### conclusion and future steps

## Author contribution

_(Please describe the contribution of each member of group)._

## References

_(List the source(s) for any data and/or literature cited in your project.  Ideally, this should be formatted using a formal citation format (MLA or APA or other, your choice!).   Multiple free online citation generators are available such as <a href="http://www.easybib.com/style">http://www.easybib.com/style</a>. **Important:** if you use **any** code that you find on the internet for your project you **must** cite it or you risk losing most/all of the points for you project.)_